Various feature extraction methods.

At the bottom of this notebook, there is code to extract all these features and compile them into a csv.

In [21]:
import cv2
import numpy as np

# Detail metrics

def edge_density(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    edges = cv2.Canny(img, 100, 200)
    density = np.sum(edges > 0) / edges.size
    return density

def laplacian_variance(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    return cv2.Laplacian(img, cv2.CV_64F).var()

from skimage import io, color
from skimage.measure import shannon_entropy

def image_entropy(image_path):
    img = io.imread(image_path)

    # Handle alpha channel if present
    if img.ndim == 3 and img.shape[2] == 4:  # RGBA
        img = img[:, :, :3]  # Drop alpha

    # Convert grayscale to 3-channel for consistency
    if img.ndim == 2:
        img = np.stack([img]*3, axis=-1)

    # Normalize to [0, 1] and convert to grayscale
    img = img.astype("float32") / 255.0
    gray = color.rgb2gray(img)

    return shannon_entropy(gray)

img = rf'dataset/mery/danbooru_866364_32a36dadb2476488304e227fbc9be19e.png'
print(image_entropy(img))
img = rf'dataset\torino\danbooru_714868_f870a467b068f93c0fc9520ae800d74b.png'
print(image_entropy(img))
img = rf'dataset\yukien\danbooru_2754192_8695c575f852b9349acb1e90a3380fd8.png'
print(image_entropy(img))

11.976495462651123
14.380766613498729
10.316068510845778


In [22]:
# Color metrics

# Hasler and Süsstrunk’s Colorfulness Metric
def colorfulness(image_path):
    img = cv2.imread(image_path)
    (B, G, R) = cv2.split(img.astype("float"))
    rg = np.absolute(R - G)
    yb = np.absolute(0.5 * (R + G) - B)

    std_rg, std_yb = np.std(rg), np.std(yb)
    mean_rg, mean_yb = np.mean(rg), np.mean(yb)

    return np.sqrt(std_rg**2 + std_yb**2) + 0.3 * np.sqrt(mean_rg**2 + mean_yb**2)

from scipy.stats import entropy

# How evenly colors are distributed. High entropy = uniform spread
# (no dominant color), low entropy = color is concentrated in few bins.
def _histogram_spread(channel, bins=32):
    hist, _ = np.histogram(channel, bins=bins, range=(0, 1), density=False)
    non_zero_bins = np.count_nonzero(hist)
    return non_zero_bins / bins  # value in [0, 1]

def _histogram_entropy(channel, bins=32):
    hist, _ = np.histogram(channel, bins=bins, range=(0, 1), density=True)
    return entropy(hist + 1e-10)  # add epsilon to avoid log(0)

def color_histogram_diversity(image_path, color_space='hsv', bins=32):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) / 255.0

    if color_space == 'hsv':
        converted = cv2.cvtColor((image * 255).astype(np.uint8), cv2.COLOR_RGB2HSV) / 255.0
    elif color_space == 'lab':
        converted = cv2.cvtColor((image * 255).astype(np.uint8), cv2.COLOR_RGB2Lab) / 255.0
    else:
        raise ValueError("Choose 'hsv' or 'lab'.")

    diversity_metrics = []
    for i in range(3):
        channel = converted[:, :, i].flatten()
        spread = _histogram_spread(channel, bins)
        ent = _histogram_entropy(channel, bins)
        diversity_metrics.append((spread, ent))

    # Combine (you can weight them differently if needed)
    avg_spread = np.mean([m[0] for m in diversity_metrics])
    avg_entropy = np.mean([m[1] for m in diversity_metrics])
    return {
        "avg_spread": avg_spread,
        "avg_entropy": avg_entropy
    }
    
img = rf'dataset/mery/danbooru_866364_32a36dadb2476488304e227fbc9be19e.png'
print(color_histogram_diversity(img))
img = rf'dataset\torino\danbooru_714868_f870a467b068f93c0fc9520ae800d74b.png'
print(color_histogram_diversity(img))
img = rf'dataset\yukien\danbooru_2754192_8695c575f852b9349acb1e90a3380fd8.png'
print(color_histogram_diversity(img))

{'avg_spread': 0.8854166666666666, 'avg_entropy': 2.225152314750004}
{'avg_spread': 0.90625, 'avg_entropy': 2.743288423144755}
{'avg_spread': 0.7395833333333334, 'avg_entropy': 2.0596846279298435}


In [23]:
# Temperature metrics

# Computes the mean and stddev of color temps in an image, from -255 (blue) to 255 (red)
def temperature_analysis(image_path):
    # Read the image
    image = cv2.imread(image_path)

    # Convert BGR to RGB (since OpenCV loads images in BGR format)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Extract Red and Blue channels
    red_channel = image[:, :, 0].astype(np.int16)
    blue_channel = image[:, :, 2].astype(np.int16)

    # Compute difference (R - B)
    diff = red_channel - blue_channel

    # Compute statistics
    temp_mean = np.mean(diff)
    temp_stddev = np.std(diff)

    return {
        "temp_mean": temp_mean,
        "temp_stddev": temp_stddev
    }

img = rf'dataset\mery\danbooru_866364_32a36dadb2476488304e227fbc9be19e.png'
print(temperature_analysis(img))
img = rf'dataset\torino\danbooru_714868_f870a467b068f93c0fc9520ae800d74b.png'
print(temperature_analysis(img))
img = rf'dataset\yukien\danbooru_2754192_8695c575f852b9349acb1e90a3380fd8.png'
print(temperature_analysis(img))

{'temp_mean': 44.861831180811805, 'temp_stddev': 44.79717545504513}
{'temp_mean': -21.360108333333333, 'temp_stddev': 41.90428196483342}
{'temp_mean': 25.560463836477986, 'temp_stddev': 23.793796918470882}


In [24]:
# Values and Contrast metrics

import numpy as np
from skimage import io, color

def grayscale_contrast_analysis(image_path):
    # Load image
    img = io.imread(image_path)

    # Handle alpha channel or grayscale input
    if img.ndim == 3 and img.shape[2] == 4:
        img = img[:, :, :3]
    elif img.ndim == 2:
        img = np.stack([img]*3, axis=-1)

    # Convert to grayscale [0, 1]
    img = img.astype("float32") / 255.0
    gray = color.rgb2gray(img)

    # Compute mean and std deviation
    mean_val = np.mean(gray)
    std_val = np.std(gray)

    return {
        "mean_gray": mean_val,
        "std_gray": std_val  # Higher = more contrast
    }

img = rf'dataset/mery/danbooru_866364_32a36dadb2476488304e227fbc9be19e.png'
print(grayscale_contrast_analysis(img))
img = rf'dataset\torino\danbooru_714868_f870a467b068f93c0fc9520ae800d74b.png'
print(grayscale_contrast_analysis(img))
img = rf'dataset\yukien\danbooru_2754192_8695c575f852b9349acb1e90a3380fd8.png'
print(grayscale_contrast_analysis(img))

{'mean_gray': 0.7696935, 'std_gray': 0.23083848}
{'mean_gray': 0.6630832, 'std_gray': 0.32251438}
{'mean_gray': 0.75510055, 'std_gray': 0.25526005}


In [25]:
# Local Binary Pattern

from skimage import io, color, feature
import numpy as np

def compute_lbp_features(image_path, radius=1, n_points=8):
    """
    Computes Local Binary Pattern (LBP) features for texture analysis.

    Parameters:
    - image_path: Path to the input image.
    - radius: Radius of circle for neighborhood.
    - n_points: Number of points considered around each pixel (typically 8 * radius).

    Returns:
    - lbp_image: LBP-transformed image.
    - histogram: Normalized histogram of LBP values (feature vector).
    """
    # Load and convert to grayscale
    img = io.imread(image_path)
    # If the image has an alpha channel (RGBA), remove it and keep just RGB
    if img.shape[2] == 4:
        img = img[:, :, :3]  # Keep only the RGB channels
    gray = color.rgb2gray(img)

    # Compute LBP
    lbp = feature.local_binary_pattern(gray, P=n_points, R=radius, method='uniform')

    # Compute normalized histogram
    n_bins = int(lbp.max() + 1)
    hist, _ = np.histogram(lbp.ravel(), bins=n_bins, range=(0, n_bins), density=True)

    return hist

img = rf'dataset/mery/danbooru_866364_32a36dadb2476488304e227fbc9be19e.png'
print(compute_lbp_features(img))
img = rf'dataset\torino\danbooru_714868_f870a467b068f93c0fc9520ae800d74b.png'
print(compute_lbp_features(img))
img = rf'dataset\yukien\danbooru_2754192_8695c575f852b9349acb1e90a3380fd8.png'
print(compute_lbp_features(img))

[0.02940498 0.05232934 0.03541513 0.09476937 0.19600554 0.13138838
 0.0466559  0.06144834 0.28041052 0.07217251]
[0.05703333 0.07615833 0.03868333 0.09896667 0.19821667 0.12821667
 0.05479167 0.08168333 0.14810833 0.11814167]
[0.01967767 0.03600629 0.01407233 0.10430818 0.19987421 0.15391509
 0.02926101 0.07389937 0.31221698 0.05676887]


C:\Users\Chien Shyong\AppData\Roaming\Python\Python311\site-packages\skimage\feature\texture.py:353: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


In [ ]:
# Compile everything
def feature_extraction(image_path):
    density = edge_density(image_path)
    variance = laplacian_variance(image_path)
    entropy = image_entropy(image_path)
    colorful = colorfulness(image_path)
    color_histogram = color_histogram_diversity(image_path)
    temperature = temperature_analysis(image_path)
    grayscale_contrast = grayscale_contrast_analysis(image_path)
    lbp_features = compute_lbp_features(image_path)

    result = [density, variance, entropy, colorful, color_histogram['avg_spread'], color_histogram['avg_entropy'],
              temperature['temp_mean'], temperature['temp_stddev'], grayscale_contrast['mean_gray'], grayscale_contrast['std_gray']]
    result += lbp_features.tolist()
    return result

image_path = rf'dataset/mery/danbooru_866364_32a36dadb2476488304e227fbc9be19e.png'
print(feature_extraction(img))

[0.0796619496855346, 776.7630187134113, 10.316068510845778, 29.93149043457331, 0.7395833333333334, 2.0596846279298435, 25.560463836477986, 23.793796918470882, 0.75510055, 0.25526005, 0.01967767295597484, 0.0360062893081761, 0.014072327044025158, 0.10430817610062892, 0.199874213836478, 0.15391509433962264, 0.029261006289308177, 0.07389937106918239, 0.31221698113207547, 0.0567688679245283]


In [ ]:
# image_path, artist_label, edge_density, laplacian_variance, shannon_entropy, hs_colorfulness, color_spread, color_entropy, 
# temp_mean, temp_stddev, gray_mean, gray_stddev, lbp_0, lbp_1, lbp_2, lbp_3, lbp_4, lbp_5, lbp_6, lbp_7, lbp_8, lbp_9

import os
import csv
from tqdm import tqdm

def insert_into_csv(image_path, artist, output_file='dataset.csv'):
    try:
        features = feature_extraction(image_path)
        data = [image_path, artist] + features

        # Append the list as a new row
        with open(output_file, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(data)
    except Exception as e:
        print(f"Error processing {image_path}: {e}")

def process_folder(folder_path, artist_label, output_file='dataset.csv'):
    image_files = [f for f in os.listdir(folder_path)]
    for image_name in tqdm(image_files):
        image_path = os.path.join(folder_path, image_name)
        insert_into_csv(image_path, artist_label, output_file)

process_folder(rf'dataset/mery', 0)
process_folder(rf'dataset/torino', 1)
process_folder(rf'dataset/yukien', 2)

 13%|█▎        | 172/1362 [00:16<01:34, 12.57it/s]c:\Python311\Lib\site-packages\PIL\Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
 50%|█████     | 682/1362 [01:11<00:52, 13.03it/s]

Error processing dataset/yukien\danbooru_6896507_01fe5370edf243fffe9a7746e1aa643d.png: the input array must have size 3 along `channel_axis`, got (564, 400, 2)
Error processing dataset/yukien\danbooru_6897930_921c3741ca19929495331a1db4e03f23.png: the input array must have size 3 along `channel_axis`, got (356, 400, 2)


 51%|█████     | 693/1362 [01:12<00:38, 17.17it/s]

Error processing dataset/yukien\danbooru_6897937_710dbba1fb7894f936660af9270fbb72.png: the input array must have size 3 along `channel_axis`, got (1322, 400, 2)
Error processing dataset/yukien\danbooru_6897938_a5f0a0e9e9f8a51ce358b3acef2e8477.png: the input array must have size 3 along `channel_axis`, got (1147, 400, 2)
Error processing dataset/yukien\danbooru_6897939_21d08a4339f7e564b00ff03775bd7a78.png: the input array must have size 3 along `channel_axis`, got (1147, 400, 2)
Error processing dataset/yukien\danbooru_6897940_65ef25a584cbf1bf0c63eadfbf2a7221.png: the input array must have size 3 along `channel_axis`, got (1147, 400, 2)
Error processing dataset/yukien\danbooru_6897941_958eeb0db38e5c17bb0aaa0adf25f9eb.png: the input array must have size 3 along `channel_axis`, got (1147, 400, 2)
Error processing dataset/yukien\danbooru_6897942_ec4342644d45d9add7fdd6c672f3c050.png: the input array must have size 3 along `channel_axis`, got (1147, 400, 2)


 52%|█████▏    | 704/1362 [01:13<00:48, 13.46it/s]

...

 68%|██████▊   | 928/1362 [01:32<00:28, 15.39it/s]

Error processing dataset/yukien\danbooru_6944864_660fe1a48c484b376299b4e087078c69.png: the input array must have size 3 along `channel_axis`, got (564, 400, 2)
Error processing dataset/yukien\danbooru_6944866_d8fc0868badbfa815c48a0f9edad1104.png: the input array must have size 3 along `channel_axis`, got (564, 400, 2)
Error processing dataset/yukien\danbooru_6944886_ebd1bacbcc0c205572191be82f584ee7.png: the input array must have size 3 along `channel_axis`, got (564, 400, 2)


 74%|███████▍  | 1006/1362 [01:38<00:28, 12.58it/s]

Error processing dataset/yukien\danbooru_6955238_f211d3d67a77c40bcfd4cd757bfe784a.png: the input array must have size 3 along `channel_axis`, got (287, 400, 2)


100%|██████████| 1362/1362 [02:17<00:00,  9.88it/s]
